In [1]:
from pdf2image import convert_from_path
import os

from pdf2image import convert_from_path
import os

# For Windows, specify poppler path
POPPLER_PATH = r"C:\Users\mason\Documents\poppler-24.08.0\Library\bin"  # Adjust this path

def convert_pdf_to_images(pdf_path, output_dir, dpi=300):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert PDF to images (with poppler path for Windows)
    images = convert_from_path(
        pdf_path,
        dpi=dpi,
        poppler_path=POPPLER_PATH  # Only needed on Windows
    )
    
    # Rest of the code remains the same
    image_paths = []
    for i, image in enumerate(images):
        output_path = os.path.join(output_dir, f'page_{i+1}.png')
        image.save(output_path, 'PNG')
        image_paths.append(output_path)
        print(f'Saved: {output_path}')
    
    return image_paths
# Example usage
if __name__ == "__main__":
    pdf_path = "test.pdf"  # Your PDF file
    output_dir = "output_images"  # Directory to save images
    
    image_paths = convert_pdf_to_images(pdf_path, output_dir)

Saved: output_images\page_1.png
Saved: output_images\page_2.png
Saved: output_images\page_3.png


In [9]:
import pytesseract
from PIL import Image, ImageEnhance, ImageOps
import pandas as pd
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'  # if needed

# Open and enhance the image, then invert it
img = Image.open('C:\\Users\\mason\\Pictures\\Screenshots\\Screenshot 2025-01-11 151305.png')
img = img.convert('L')  # Convert to grayscale
enhancer = ImageEnhance.Contrast(img)
img_enhanced = enhancer.enhance(2.0)  # Increase contrast
img_inverted = ImageOps.invert(img_enhanced)

# Upscale the inverted image (e.g., 2x)
width, height = img_inverted.size
img_upscaled = img_inverted.resize((width * 2, height * 2), Image.Resampling.LANCZOS)

# Save the inverted, upscaled image
os.makedirs('td', exist_ok=True)
processed_image_path = os.path.join('td', 'inverted_upscaled_screenshot.png')
img_upscaled.save(processed_image_path)
print(f'Saved processed image to: {processed_image_path}')

# First pass (text) - no whitelist
config_text = r"--psm 6 --oem 3"
df_ocr_text = pytesseract.image_to_data(
    img_upscaled,
    config=config_text,
    lang='eng',
    output_type='data.frame'
)
df_ocr_text = df_ocr_text.dropna(subset=['text'])

# Group by line_num to get lines of text
lines_text = df_ocr_text.groupby('line_num')['text'].apply(list).reset_index()

# Identify which row is the column header (contains words 'Bid' and 'Ask')
header_row = lines_text[lines_text['text'].apply(lambda x: 'Bid' in x and 'Ask' in x)]
if not header_row.empty:
    header_words = header_row.iloc[0]['text']
    header_idx = header_row.iloc[0]['line_num']
else:
    # fallback if no header row is found
    header_words = []
    header_idx = None

# Identify data lines (any line other than the header line)
data_lines = lines_text[lines_text['line_num'] != header_idx]

# Prepare a list to hold final rows
results = []

# Second pass (numeric) - whitelist digits and punctuation
# In a real scenario, you'd crop or otherwise isolate numeric columns:
cropped_img = img_upscaled  # Placeholder if you haven't actually cropped
config_nums = r"--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789.,"
numeric_df = pytesseract.image_to_data(
    cropped_img,
    config=config_nums,
    lang='eng',
    output_type='data.frame'
)
numeric_df = numeric_df.dropna(subset=['text'])

# (Optionally, for illustration, show or group numeric data)
numeric_lines = numeric_df.groupby('line_num')['text'].apply(list).reset_index()

# Map the text data lines to the header
for i, row in data_lines.iterrows():
    data_words = row['text']
    if len(data_words) == len(header_words):
        data_dict = dict(zip(header_words, data_words))
        # Attempt to convert certain columns to floats
        numeric_cols = ['Bid', 'Ask', 'Price', 'TPrice', 'Volume', 'OI', 'Strike']
        for col in numeric_cols:
            if col in data_dict:
                try:
                    data_dict[col] = float(data_dict[col])
                except ValueError:
                    data_dict[col] = None
        results.append(data_dict)

# Create dataframe from the extracted rows
df_result = pd.DataFrame(results)
print(df_result)


Saved processed image to: td\inverted_upscaled_screenshot.png
  Symbol   Bid   Ask  Price  TPrice  Volume      OI NS
0   NVDA  2.76  3.15   2.93     0.0  7676.0  2953.0  W


In [10]:
df_result.head()

,Symbol,Bid,Ask,Price,TPrice,Volume,OI,NS
0,NVDA,2.76,3.15,2.93,0.0,7676.0,2953.0,W
